In [4]:
repl.load(os.read(os.pwd/'git/'src/'main/'scala/"import.sc"))
// repl.load(os.read(os.pwd/"a.sc"))
repl.load(os.read(os.pwd/'git/'src/'main/'scala/"a.scala"))
// repl.load(os.read(os.pwd/'git/'diss/'src/'main/'scala/"b.sc"))


Compiling /home/jovyan/data/(console)
Compiling /home/jovyan/data/(console)


In [5]:
import $ivy.`com.timgroup::iterata:0.1.7`

import $ivy.$

In [6]:
import Diss._
import UaMeta._

import Diss._
import UaMeta._

In [7]:
  lazy val badFiles: Set[(String, String)] =
    scala.io.Source.fromFile("all.ban", "UTF-8").getLines.map { line =>
      val split = line.split("\t")
      split.head -> split(1)
    }.toSet


lazy val graphBad: Map[Int, Vector[Int]] = {
    scala.io.Source.fromFile("ua.hash.7").getLines.flatMap { line =>
      val split = line.split("\t")

      val snd = split(1).toInt
      val fst = split(0).toInt
      if (getUaMetaById(fst).year == getUaMetaById(snd).year) {
        Vector(snd -> fst, fst -> snd)
      } else {
        Vector(snd -> split(0).toInt)
      }
    }.toVector.groupBy(_._1).mapValues(_.map(_._2)).withDefaultValue(Vector.empty)
  }

  def extractNumPm(str: String): List[String] = {
    val main = str.replaceAll("[.,]", "").map(c => if (c.isDigit || c == 177.toChar) c else ' ')
    val split = main.split(177.toChar)
    val list = split.map(x => x.split(" ").filter(_.nonEmpty))
    (for {i <- 1 to list.size - 1
          if list(i - 1).size >= 2 && list(i).size >= 2 && list(i - 1).last.length > 1 && list(i).head.length > 1
          } yield s"${list(i - 1).last}/${list(i).head}").toList
  }

  def uaFileHandleToNum(work: String): List[(List[String], Int)] = {
    import scala.util.Try
    val numbers: List[(List[String], Int)] = Try {

      val a = work.split(12.toChar)

      val zip: Array[(String, Int)] = a.zipWithIndex
      val rev = zip.reverse

      val sub: List[(String, Int)] = zip.toList.sliding(3).toList.flatMap { array =>
        if (array(1)._1.toUpperCase.contains("ТАБЛИЦ")) {
          array
        } else {
          List.empty
        }
      }.distinct.sortBy(_._2)

      val res: List[(List[String], Int)] = sub.map(x => (extractNumPm(x._1), x._2))

      res.filter(_._1.size >= 10)
    }.getOrElse(List())

    numbers
  }

  def getDirPath(id: String): String = {
    s"txt/${id.takeRight(3).toInt}/$id/"
  }

  def readById(id: String): String = {
    val dir = getDirPath(id)
    val all: Vector[String] = FileUtils.traverse(dir).filter(_.endsWith(".txt")).filterNot(x => isAutoRefUA(x)).filterNot(x => badFiles.contains(id -> x.split("/").last)).map(read)
    all.mkString(12.toChar.toString)
  }


badFiles: Set[(String, String)] = [lazy]
graphBad: Map[Int, Vector[Int]] = Map(
  53767 -> Vector(114232, 24948, 23846, 43926, 56826, 51220, 40123, 24691, 45608),
  32571 -> Vector(20530, 29378),
  106805 -> Vector(108741, 120672, 96832, 50234, 100430),
  88533 -> Vector(
    28978,
    62439,
    43322,
    60632,
    21703,
    42881,
    74229,
    73755,
    72794,
    10526,
    35083,
    70040,
    54857,
    40287,
    46139,
    117596,
    31987,
    83211,
    53891,
    36686,
    27736,
    88532,
    22001,
    51223,
    30219,
    40281,
    44176,
    62153,
    69770,
    113433,
    90583,
    46995,
    84631,
...
defined function extractNumPm
defined function uaFileHandleToNum
defined function getDirPath
defined function readById

In [8]:
  def splitPair(value: String): (String, String) = {
    val split = value.split("/")
    split.head -> split.last
  }


  def comparePair(fst: String, snd: String): Boolean = {
    val (fa, fb) = splitPair(fst)
    val (sa, sb) = splitPair(snd)


    val lev1 = editDist(fa, sa)
    val lev2 = editDist(fb, sb)

    //     val ff1 = scala.util.Try{s"${fa.take(1)}.${fa.drop(1)}".toDouble}.getOrElse(0.1)
    //     val ss1 = scala.util.Try{s"${sa.take(1)}.${sa.drop(1)}".toDouble}.getOrElse(0.1)


    (fst, snd, lev1, lev2) match {
      case (f, s, l1, l2) if l1 + l2 <= 1 && fst.length == snd.length => true

      //       case (f, s, l1, l2) if l1 + l2 <= 1 && fst.length != snd.length => true
      //       case (f, s, l1, l2) if l1 <= 1 && l2 <= 1 => true
      //       case (f, s, l1, l2) if scala.math.abs(ff1 - ss1) / ss1 <= 0.2 && l2 <= 1 => true
      //      case (f, s, mi, ma, diff, l) if min >= 5 && l <= 2 => true
      //      case (f, s, mi, ma, diff, l) if min == 2 && max == 2 && (f.startsWith("0") || f.startsWith("1")) && (s.startsWith("0") || s.startsWith("1")) => true
      //      case (f, s, mi, ma, diff, l) if (!f.startsWith("0") && !s.startsWith("0")) && scala.math.abs(f.toDouble - s.toDouble) / f.toDouble < 0.2 => true
      case _ => false
    }
  }

  //   def compareNgram(fst: List[String], snd: List[String]): Boolean = {
  //     (fst zip snd).forall(x => comparePair(x._1, x._2))
  //   }


  //   def nonTrivialVector(value: List[String]): Boolean = {
  //     value.distinct.size == value.size && value(0) >= value(1) && value.forall(_.length >= 2) && value.exists(_.length >= 3)
  //   }

  def similarity(list: List[(Int, Int)]): Double = {
    val s = list.sortBy(_._1).toVector
    var res = 0
    for {i <- s.indices
         j <- i + 1 until s.size
         if s(i)._2 > s(j)._2
         } {
      res = res + 1
    }

    1 - (res.toDouble / (s.size * s.size))
  }

  def similarityLinear(list: List[(Int, Int)]): Double = {
    val s = list.sortBy(_._1).toVector
    var res = 0
    for {i <- 0 until s.size - 2
         if s(i)._2 > s(i + 1)._2
         } {
      res = res + 1
    }

    1 - (res.toDouble / (s.size - 1))
  }


  def compareTwoTables(fst: List[String], snd: List[String]): (Int, Int, Int, Int, Double, Double, Int, Int, Int, Int) = {
    val fstU = fst.zipWithIndex
    val sndSet = snd.toSet
    val sndZip = snd.zipWithIndex

    val fuzzy: List[(Int, Int)] = fstU.filter(x => !sndSet.contains(x._1)).flatMap {
      f => sndZip.find(s => comparePair(f._1, s._1)).map(s => (f._2, s._2))
    }

    val eq: List[(Int, Int)] = fstU.filter { case (f, idx) =>
      sndSet.contains(f)
    }.flatMap(f => sndZip.find(x => f._1 == x._1).map(x => (f._2, x._2)))

    val tuples: List[(Int, Int)] = fuzzy ++ eq

    val f1 = tuples.map(p => fst(p._1)).distinct.size
    val s1 = tuples.map(p => snd(p._2)).distinct.size

    (fuzzy.size, eq.size, fst.size, snd.size, similarity(tuples), similarityLinear(tuples), fst.distinct.size, snd.distinct.size, f1, s1)
  }

  // (page, page)
  def compareTwoWorks(fst: List[(List[String], Int)], snd: List[(List[String], Int)]): List[(Res, Res, Res, Res, Res, Int, Double, Double, Int, Int, Int, Int)] = {
    for {
      fstPage <- fst
      sndPage <- snd
      s2 = compareTwoTables(fstPage._1, sndPage._1)
      if s2._1 + s2._2 >= 15 && (s2._1 + s2._2 + 0.1) / s2._3 >= 0.5 && s2._5 >= 0.95 // && s2._2 > 5 && s2._2 >= s2._1 / 2
    } yield (fstPage._2, sndPage._2, s2._1, s2._2, s2._3, s2._4, s2._5, s2._6, s2._7, s2._8, s2._9, s2._10)
  }


defined function splitPair
defined function comparePair
defined function similarity
defined function similarityLinear
defined function compareTwoTables
defined function compareTwoWorks

In [ ]:
import com.timgroup.iterata.ParIterator.Implicits._
val iter: Iterator[String] = uaMetaVec.filter(x => !x.spec.startsWith("14.0")).sortBy(_.id)(Ordering[String].reverse).iterator.par().flatMap { mainMeta =>
    val longStrId = mainMeta.id
    val mainId: Int = idUaRevMap(longStrId)
    val neigh = graphBad(mainId)
    if (neigh.nonEmpty) {
      val mainText = readById(longStrId)
      val mainTables = uaFileHandleToNum(mainText)
      neigh.par.flatMap { otherBadId =>
        val otherMeta = getUaMetaById(otherBadId)
        val otherLongStrId = idUaMap(otherBadId)
        val otherText = readById(otherLongStrId)
        val otherTables = uaFileHandleToNum(otherText)

        val tuples = compareTwoWorks(mainTables, otherTables)

        val header = List[String](mainMeta.id, mainMeta.year.toString, mainMeta.toString, otherMeta.id, otherMeta.year.toString, otherMeta.toString)

        if (tuples.nonEmpty) {
          List(header.mkString("\t")) ++
            tuples.map {
              case (page1, page2, f, eq, total1, total2, sim, simL, fstUniq, sndUniq, f1, s1) =>
                val list = List[String]("", "", "", "", "", "", "page: " + (page1 + 1).toString, "page: " + (page2 + 1).toString,
                  "fuzzy: " + f.toString, "eq: " + eq.toString, "total1: " + total1.toString, "total2: " + total2.toString, "sim: " + sim.toString, "simL:" + simL.toString, "fstUniq: " + fstUniq.toString, "sndUniq: " + sndUniq.toString,
                  "dSize1: " + f1.toString, "dSize2: " + s1.toString)
                list.mkString("\t")
            }
        } else {
          Nil
        }
      }
    } else {
      Nil
    }
  }

  FileUtils.write("uaTablePM.others.csv", iter)


In [ ]:
uaMetaVec.filter(_.spec.startsWith("14.0")).sortBy(_.id)(Ordering[String].reverse).take(1000).iterator

In [ ]:
uaMetaVec.filter(_.spec.startsWith("14.01.01")).size

In [ ]:
uaMetaVec.filter(_.spec.startsWith("14.0")).size